In [1]:

# -*- coding: utf-8 -*-
try:
    from src.utils import *
except:
    from utils import *
    
from dotenv import load_dotenv
load_dotenv(".env")

print('読み込み完了')

for target_dir in [r'image\temp_concat_image_dir',r'image\temp_tweet_image_dir',r'image\temp_image']:
    try:
        shutil.rmtree(target_dir)
    except:
        pass
    finally:
        os.mkdir(target_dir)
def get_concat_v_blank(im_list, color=(0, 0, 0)):
    dst = Image.new('RGB', (1000,max(im.height for im in im_list)), color)
    for i,im in enumerate(im_list):
        if i == 0:
            dst.paste(im, (0, 0))
        else:
            dst.paste(im, (0, im_list[i-1].height))
    return dst

def get_concat_h_blank(im_list, color=(0, 0, 0)):
    
    dst = Image.new('RGB', (1000*len(im_list),max(im.height for im in im_list)), color)
    for i,im in enumerate(im_list):
        print(i)
        if i == 0:
            dst.paste(im, (0, 0))
        else:
            dst.paste(im, ((i)*1000,0))
    return dst


# アスペクト比を固定して、幅が指定した値になるようリサイズする。
def scale_to_width(img_path, width):
    img = Image.open(img_path)
    height = round(img.height * width / img.width)
    return img.resize((width, height))


def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                      for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

scraping = PledgeScraping()
target_day_number = 1
scraping.add_target_date(target_day_number)
browser = scraping.login_scraping_site('chubu')
prefecture_name_and_number_dict = scraping.get_prefecture_name_and_number_dict()

#pledge_imageフォルダの中身をリスト化
pledge_image_path_list = []
for pledge_image_path in glob.glob(r'image\pledge_image\*'):
    pledge_image = str(pledge_image_path).replace('image\\pledge_image\\','').replace('.jpg','')
    pledge_image_path_list.append(pledge_image)
    
rank_color_dict:dict[str:tuple] = {'S':(255,0,0),'A':(0,128,0),'B':(255,255,0),'C':(0,0,255),'・':(255,255,255)}
prefecture_wp_url_dict ={'愛知県':'http://bit.ly/3G54gt7',
                         '静岡県':'http://bit.ly/3Kmxupy',
                        '岐阜県':'http://bit.ly/3nAdhUF',
                        '三重県':'http://bit.ly/3U04cAn'}


for prefecture_name in ['愛知県','三重県','静岡県','岐阜県',]:

    for target_dir in [r'image\temp_concat_image_dir',r'image\temp_tweet_image_dir',r'image\temp_image']:
        try:
            shutil.rmtree(target_dir)
        except:
            pass
        finally:
            os.mkdir(target_dir)

    blog = Blog()
    twitter = UtilsTwitterClass()
    print(prefecture_name,target_day_number)
    blog.add_target_date (target_day_number)
    blog.prefecture_name = prefecture_name
    print(prefecture_name)
    scraping_target_date_pledge_df = scraping.read_target_date_pledge_df(prefecture_name)
    read_convert_parlar_name_df = scraping.read_convert_parlar_name_df()
    merged_syuzai_pledge_df = scraping.generate_merged_syuzai_pledge_df()
    save_main_image_path_list = scraping.create_pledge_main_images()

    title = f"【{blog.prefecture_name}】{blog.target_date_string_jp} パチンコスロットイベント取材まとめ"
    input_by_pledge_text = f'\n<h2>{scraping.prefecture_name} {scraping.target_date_string_jp} 媒体別取材まとめ</h2>\n'
    pledge_name_list:list[str] = list(merged_syuzai_pledge_df['媒体名'].unique())
    for pledge_name in ['リニューアルオープン','フリー','その他取材・企画','その他','未調査']:
        try:
            pledge_name_list.remove(pledge_name)
            pledge_name_list.append(pledge_name)
        except:
            pass
    error_pledge_name_list = []

    #リストの先頭に旧イベを追加
    if '旧イベ' in pledge_name_list:
        pledge_name_list.remove('旧イベ')
        pledge_name_list.insert(0, '旧イベ')

    if 'ドリスロ' in pledge_name_list:
        pledge_name_list.remove('ドリスロ')
        pledge_name_list.insert(2, 'ドリスロ')

    for pledge_name in pledge_name_list:
        extract_pledge_name_df = merged_syuzai_pledge_df[merged_syuzai_pledge_df['媒体名'] == pledge_name]
        display(extract_pledge_name_df)
        concat_generate_pledge_image_list:list[Image] = []
        if pledge_name in pledge_image_path_list:
            print(pledge_name)
            image = Image.open(fr'image\pledge_image\{pledge_name}.jpg')
            concat_generate_pledge_image_list.append(image)
        else:
            print('error',pledge_name)
            #取材別のヘッダー画像を作成
            image = Image.open(r'image\pledge_image\eva_pledge_banner.jpg')
            draw = ImageDraw.Draw(image)
            #フォントを指定する（フォントファイルはWindows10ならC:\\Windows\\Fontsにあります）
            font_path = r"font\NotoSerifJP-Black.otf"
            font = ImageFont.truetype(font_path, size=80)
            draw.multiline_text((180,10), f'{pledge_name}\n{blog.target_date_string_jp} {prefecture_name}', fill=(255,255,255), font=font,spacing=10)
            #画像を保存する
            image.save(fr'image\temp_image\test_{blog.target_date_string_jp}_{pledge_name}.jpg')
            concat_generate_pledge_image_list.append(image)
        #write_pledge_text = ''

        for rank in ['S','A','B','C','・']:
            extract_syuzai_name_df  = extract_pledge_name_df[extract_pledge_name_df['取材ランク'] == rank]
            print(rank)
            if len(extract_syuzai_name_df) == 0:
                continue
            display(extract_syuzai_name_df)
            write_pledge_text = ''
            for syuzai_name in extract_syuzai_name_df['取材名'].unique():
                print('1',syuzai_name)
                write_pledge_text += '\n\n' + syuzai_name 
                extract_parlar_name_df  = extract_pledge_name_df[extract_pledge_name_df['取材名'] == syuzai_name]
                pre_pledge_name = ''
                image = Image.open(fr'image\image_source\eva_board.jpg')
                for i,row in extract_parlar_name_df.iterrows():
                    #print(row)
                    if pre_pledge_name != row['取材名']:
                        write_pledge_text += f'\n┗公約 : {row["公約内容"]}'
                        write_pledge_text += f'\n　 ◆{row["店舗名"]}'
                    else:
                        write_pledge_text += f'\n　 ◆{row["店舗名"]}'
                        
                    pre_pledge_name = row['取材名']
                draw = ImageDraw.Draw(image)
                #フォントを指定する（フォントファイルはWindows10ならC:\\Windows\\Fontsにあります）
                font_path = r"font\LightNovelPOPv2.otf"
                font = ImageFont.truetype(font_path, size=50)
                draw = ImageDraw.Draw(image)
                draw.multiline_text((50, -100), write_pledge_text, fill=(255,255,255),align='left',font=font, spacing=15)
                save_image_jpg = r'image\temp_image\\' +fr'{prefecture_name}_{blog.target_date_string_jp}_{pledge_name}_{rank}.jpg'
                image_length = write_pledge_text.count('\n')
                print(image_length)
                print('length', image_length)
                length_croped = image_length * 70  # 500
                print(length_croped)
                im_crop = image.crop((0, 0, 1000, length_croped))
                im_crop.save(save_image_jpg, quality=100)
                
                line_width = 20  # line width（枠線一本の幅）
                waku_color = rank_color_dict[rank] # 枠線の色（RGB）

                # 元になる画像からImageオブジェクトを作成
                src_im = Image.open(save_image_jpg)
                sw, sh = src_im.size  # 元の画像のサイズを取得

                # キャンバスを作成
                # この画像に枠線と元の画像を合成する
                cw, ch = sw + line_width * 2, sh + line_width * 2  # キャンバスのサイズを元の画像から生成
                canvas_im = Image.new('RGB', (cw, ch))

                # キャンバスのImageDrawオブジェクトを作成
                canvas = ImageDraw.Draw(canvas_im)

                # キャンバスを単色で塗りつぶす
                canvas.rectangle([(0, 0), (cw, ch)], fill=waku_color)

                # キャンバスの画像に元の画像を貼り付け
                canvas_im.paste(src_im, (line_width, line_width))
                canvas_im.save(save_image_jpg)
            concat_generate_pledge_image_list.append(canvas_im)
                #break
            #break
        # if len(extract_pledge_name_df ) != 0:
        #     # print('\n■',baitai,sep='')
        #     write_pledge_text += '<h3>' + string_date_only + ' ' + todoufuken_kanji + ' スロット ' + baitai + '</h3>' + f'\n<img src="http://slotana777.com/wp-content/uploads/2021/04/{error_baitai}.jpg" alt="{error_baitai} タイトル画像" width="1000" height="400" class="alignnone size-full " />\n'
        # else:
        #     pass
        #break

        get_concat_v_multi_resize(concat_generate_pledge_image_list).save(fr'image\temp_concat_image_dir\{prefecture_name}_{blog.target_date_string_jp}_{pledge_name}.jpg')

    #ディレクトリの中を時間順にリストで取得する
    concat_temp_image__list = []
    for file in sorted(glob.glob(r'image\temp_concat_image_dir\*.jpg'), key=os.path.getmtime):
        print(file)
        #画像を開く
        image = Image.open(file)
        concat_temp_image__list.append(image)
        #画像をリサイズする

    temp_list = []
    self_image_count_num = 0
    for y in range(len(concat_temp_image__list)):
        self_image_count_num += 1
        temp_list.append(concat_temp_image__list[y])
        if len(temp_list) == 6:
            self_image_count_num += 1
            print('ここで関数実行とselfに挿入',temp_list)
            save_image_jpg = r'image\temp_tweet_image_dir\pillow_concat_h_blank_' +fr'{self_image_count_num}.jpg'
            get_concat_h_blank(temp_list, (169, 206, 255)).save(save_image_jpg)
            scale_to_width(save_image_jpg, 3800).save(save_image_jpg)
            twitter.image_path_list.append(save_image_jpg)
            temp_list = []
            
            if self_image_count_num == 4:
                break

    if len(temp_list) != 0:
        self_image_count_num += 1
        print('ここで関数実行とselfに挿入',temp_list)
        save_image_jpg = r'image\temp_tweet_image_dir\pillow_concat_h_blank_' +fr'{self_image_count_num}.jpg'
        get_concat_h_blank(temp_list, (169, 206, 255)).save(save_image_jpg)
        scale_to_width(save_image_jpg, 3800).save(save_image_jpg)
        twitter.image_path_list.append(save_image_jpg)
    else:
        pass
    
    tweet_text:str = f'{prefecture_name} '+ scraping.target_date_string_jp + '明日の媒体別まとめ' 

    for i,parlar_name in enumerate(merged_syuzai_pledge_df['店舗名'].unique()):
        tweet_text += '\n' + parlar_name
        if i > 3:
            break
    tweet_text += '\n' + '他' + str(len(merged_syuzai_pledge_df['店舗名'].unique()) - 4) + '店舗'
    tweet_text += '\n' + '詳細はこちら' + '\n' + prefecture_wp_url_dict[prefecture_name]
    print(tweet_text)

    twitter.tweet_text = tweet_text
    twitter.id =os.getenv('TWITTER_ID')
    twitter.pw =os.getenv('TWITTER_PW')
    twitter.twitter_login()
    twitter.post_tweet()

    break

読み込み完了


In [7]:
twitter.image_path_list = twitter.image_path_list[:4]

['image\\temp_tweet_image_dir\\pillow_concat_h_blank_6.jpg',
 'image\\temp_tweet_image_dir\\pillow_concat_h_blank_12.jpg',
 'image\\temp_tweet_image_dir\\pillow_concat_h_blank_18.jpg',
 'image\\temp_tweet_image_dir\\pillow_concat_h_blank_24.jpg',
 'image\\temp_tweet_image_dir\\pillow_concat_h_blank_26.jpg']

In [10]:
for target_dir in [r'image\temp_concat_image_dir',r'image\temp_tweet_image_dir',r'image\temp_image']:
    try:
        shutil.rmtree(target_dir)
    except:
        pass
    finally:
        os.mkdir(target_dir)

In [10]:
twitter = UtilsTwitterClass()
vars(twitter)

{'tomorrow_8numbers_string': '20230410',
 'tomorrow_date_stinrg_jp': '4/10(月)',
 'tomorrow_date_string_sql': '2023-04-10',
 'yesterday': datetime.date(2023, 4, 8),
 'tomorrow': datetime.date(2023, 4, 10),
 'yesterday_date_string': '4/8(土)',
 'id': '',
 'pw': '',
 'tweet_text': '',
 'image_path_list': [],
 'main_tweet_text': '',
 'project_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site',
 'image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image',
 'tweet_footer_text': '\n\n#XXXX 20230408\n\n',
 'created_image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image\\image',
 'servise_account_json_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\config\\twitteranalytics-jsonsercretkey.json'}

In [ ]:


except Exception as e :
    blog.post_line(f'\n{e}')

finally:
    try:
        target_dir = r'image\temp_image'
        shutil.rmtree(target_dir)
    except:
        pass
    finally:
        os.mkdir(target_dir)
        error_pledge_name_list = list(set(blog.error_pledge_name_list))
        blog.post_line(f'\nエラー媒体名一覧\n{error_pledge_name_list}')
        blog.post_line(f'全ての処理が終わりました。')

In [ ]:

baitai_list = ['スロパチ','天草ヤスヲ','ホールサーチマン','ホール攻略','Gooパチ','パチスロ必勝本','スクープTV','ジャンバリ','trista','dorislo','アツ姫','爆ガチ！']

    for baitai in baitai_list:
        context = f'''{h1_text} 取材まとめ'''
        image_context  = f'{h1_text} 媒体別まとめ'
        todoufuken = h1_text.split('  ')[0]  
        
        tokyou_syuzai_df_a = tokyou_syuzai_df_nokori = tokyou_syuzai_df_defalut [tokyou_syuzai_df_defalut ['baitai'] == baitai]
        #display(tokyou_syuzai_df_a)
        tokyou_syuzai_df = tokyou_syuzai_df[tokyou_syuzai_df['baitai'] != baitai]
        previous_syuzai = ''
        #print('\n'  ,baitai)
        for syuzai in tokyou_syuzai_df_a.itertuples():
            print(syuzai)
        #syuzai_list.append(syuzai)
            if syuzai[1] != previous_syuzai: 
                #print('\n\n■',syuzai[1],'\n   ★',syuzai[3],'\n   ┗',syuzai[4],sep='')
                context += '\n■' + str(syuzai[1])+ '\n    ★' + str(syuzai[3]) + '\n    ┗' + str(syuzai[4]) + '\n'
                previous_syuzai = syuzai[1]
            else:
                print('未調査！！！！',syuzai)
                context += '\n    ★' + str(syuzai[3])+ '  【' + str(syuzai[2]) + '】'+'\n   ┗' + str(syuzai[4]) + '\n'
                previous_syuzai = syuzai[1]
                pass
                pass
                #print('   ★',syuzai[3],'\n   ┗',syuzai[4],sep='')

        length = context.count('\n')
        if length == 0:
            context = F'''{h1_text} 取材まとめ
            
            取材予定はありませんでした'''
            length = 4
        recommend_image(context,length)

        im1 = Image.open(f"C:\\Users\\81801\\Desktop\\全媒体jpg\\{baitai}.jpg")
        im2 = Image.open(f"C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\{todouhuken}_syuzai_reoort_{tomorrow_str}.jpg")
        get_concat_v(im1, im2).save(f"C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai}.jpg")
        os.remove(f"C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\{todouhuken}_syuzai_reoort_{tomorrow_str}.jpg")


    image_context = f'{h1_text} 取材まとめ'
    for syuzai in tokyou_syuzai_df.itertuples():
        print(syuzai)
        #syuzai_list.append(syuzai)
        if syuzai[1] != previous_syuzai: 
            #print('\n\n■',syuzai[1],'\n   ★',syuzai[3],'\n   ┗',syuzai[4],sep='')
            image_context += '\n■' + str(syuzai[1])+ '\n    ★' + str(syuzai[3])+ ' 【' + str(syuzai[2]) + '】'+'\n    ┗' + str(syuzai[4]) + '\n'
            previous_syuzai = syuzai[1]
        else:
            print('未調査！！！！',syuzai)
            image_context += '\n    ★' + str(syuzai[3])+ ' 【' + str(syuzai[2]) + '】'+'\n    ┗' + str(syuzai[4]) + '\n'
            previous_syuzai = syuzai[1]
            pass
            #print('   ★',syuzai[3],'\n   ┗',syuzai[4],sep='')

        length = image_context.count('\n')
        if length == 0:
            image_context = F'''{h1_text} 取材まとめ
            
            取材予定はありませんでした'''
            length = 4
    recommend_image(image_context,length)



    im1 = Image.open(f'C:\\Users\\81801\\Desktop\\全媒体jpg\\未調査.jpg')
    im2 = Image.open(f"C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\{todouhuken}_syuzai_reoort_{tomorrow_str}.jpg")
    get_concat_v(im1, im2).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_未調査.jpg')
    #os.remove(f'C:\\Users\\81801\\Desktop\\全媒体jpg\\eva_board_{tomorrow_url}.jpg')



    def get_concat_h_multi_blank(im_list):
        _im = im_list.pop(0)
        for im in im_list:
            _im = get_concat_h_blank(_im, im)
        return _im

    im1 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_スロパチ.jpg')
    im2 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_天草ヤスヲ.jpg')
    im3 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_ホールサーチマン.jpg')
    im4 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_ホール攻略.jpg')

    get_concat_h_multi_blank([im1, im2]).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_天草ヤスヲ_スロパチ_ホール_ホールサーチマン_ホール攻略.jpg')

    im1 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_Gooパチ.jpg')
    im2 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_パチスロ必勝本.jpg')
    im3 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_アツ姫.jpg')
    im4 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_爆ガチ！.jpg')

    get_concat_h_multi_blank([im1, im2, im3,im4]).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_Gooパチ_パチスロ必勝本_アツ姫_爆ガチ！.jpg')

    im1 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_ジャンバリ.jpg')
    im2 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_スクープTV.jpg')
    im3 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_dorislo.jpg')
    im4 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_trista.jpg')

    get_concat_h_multi_blank([im1, im2, im3,im4]).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_ジャンバリ_スクープTV_dorislo_trista.jpg')

    from PIL import Image

    def get_concat_v(im1, im2):
        dst = Image.new('RGB', (im1.width, im1.height + im2.height))
        dst.paste(im1, (0, 0))
        dst.paste(im2, (0, im1.height))
        return dst


    for number in [0,4,8]:
        im1 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+0]}.jpg')
        im2 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+1]}.jpg')
        get_concat_h_blank(im1, im2,(0, 64, 128)).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+0]}_{baitai_list[number+1]}.jpg')

        im3 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+2]}.jpg')
        im4 = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+3]}.jpg')
        get_concat_h_blank(im3, im4,(0, 64, 128)).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+2]}_{baitai_list[number+3]}.jpg')

        im1_ = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+0]}_{baitai_list[number+1]}.jpg')
        im2_ = Image.open(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_{baitai_list[number+2]}_{baitai_list[number+3]}.jpg')

        get_concat_v(im1_,im2_).save(f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_comleted_{number}.jpg')


    baitai_coutnt_dict = tokyou_syuzai_df_defalut['baitai'].value_counts().to_dict()

    baitai_list = ['スロパチ','天草ヤスヲ','ホールサーチマン','ホール攻略','Gooパチ','パチスロ必勝本','スクープTV','ジャンバリ','trista','dorislo','アツ姫','爆ガチ！']

    for baitai in baitai_list:
        try:
            value = baitai_coutnt_dict[f'{baitai}']
            print(baitai,value,'件')
        except:
            print(baitai,'0')
            
    print('その他媒体・未調査')
    tokyou_syuzai_df['baitai'].value_counts().sum()


    context = ''
    baitai_list = ['スロパチ','天草ヤスヲ','ホールサーチマン','ホール攻略','Gooパチ','パチスロ必勝本','スクープTV','ジャンバリ','trista','dorislo','アツ姫','爆ガチ！']
    tokyou_syuzai_df = tokyou_syuzai_df_defalut.sort_values('syuzai')
    for baitai in baitai_list:
        tokyou_syuzai_df_a = tokyou_syuzai_df[tokyou_syuzai_df['baitai'] == baitai]
        if len(tokyou_syuzai_df_a) != 0:
            #print('\n■',baitai,sep='')
            context += '\n\n◆' + baitai 
        else:
            pass

        past_syuzai = ''
        past_syuzai_name = ''
        for df_row in tokyou_syuzai_df_a.itertuples():
            #print(syuzai)
            if df_row[1]!= past_syuzai:
                if df_row[3] !=past_syuzai_name :
                    context += ',\n' + df_row[3] +',\n・' + df_row[1]
                else:
                    context += ',\n・' + df_row[1]

            past_syuzai = df_row[1]
            past_syuzai_name  = df_row[3]
                
            
    tenpo_only_text =  context.split(',')
    print(context)

    tenpo_only_text

    path_list = [f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_comleted_0.jpg',f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_comleted_4.jpg',f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_comleted_8.jpg',f'C:\\Users\\81801\\Desktop\\tomorrow_report\\3todoufuken_image_report\\eva_board_{todoufuken}_{tomorrow_url}_未調査.jpg' ]

    number = 1
    tenpo_only_text_completed = tenpo_only_text
    for text in tenpo_only_text_completed:
        print(len(text))
        print(number)
        print(text)



    context_1 = f'{h1_text} スロットイベント媒体別まとめ１\nその他・全取材はブログで↓\nhttps://bit.ly/3u5tZJN'
    context_2 = f'{h1_text} スロットイベント媒体別まとめ２'
    context_3 = f'{h1_text} スロットイベント媒体別まとめ３'
    context_4 = f'{h1_text} スロットイベント媒体別まとめ４'
    context_5 = f'{h1_text} スロットイベント媒体別まとめ５'
    context_6 = f'{h1_text} スロットイベント媒体別まとめ６'
    context_7 = f'{h1_text} スロットイベント媒体別まとめ７'
    pop_list = 0
    for tenpo_text in tenpo_only_text_completed:
        if len(context_1) < 120:
            context_1 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
        else:
            if len(context_2) < 120:
                context_2 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
            else:
                if len(context_3) < 120:
                    context_3 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
                else:            
                    if len(context_4) < 120:
                        context_4 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
                    else:
                        if len(context_5) < 120:
                            context_5 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
                        else:
                            if len(context_6) < 120:
                                context_6 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
                            else:
                                if len(context_7) < 120:
                                    context_7 += tenpo_text.replace('・・◆','◆').replace('・・','・').replace('・◆','◆')
                                else:
                                    pass


    print(context_1)
    print('次2')
    print(context_2)
    print('次3')
    print(context_3)
    print('次4')
    print(context_4)
    print('次5')
    print(context_5)
